In [2]:
from datasets import load_dataset

ds = load_dataset("Falah/Alzheimer_MRI")

README.md: 0.00B [00:00, ?B/s]

c:\Users\sanhi\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sanhi\.cache\huggingface\hub\datasets--Falah--Alzheimer_MRI. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


data/train-00000-of-00001-c08a401c53fe53(…):   0%|          | 0.00/22.6M [00:00<?, ?B/s]

data/test-00000-of-00001-44110b9df98c558(…):   0%|          | 0.00/5.65M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5120 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1280 [00:00<?, ? examples/s]

In [3]:
f_5 = ds['train'][:5]  # Assuming you want from the training split
print(f_5)

{'image': [<PIL.JpegImagePlugin.JpegImageFile image mode=L size=128x128 at 0x1C83CAD4B50>, <PIL.JpegImagePlugin.JpegImageFile image mode=L size=128x128 at 0x1C83E661930>, <PIL.JpegImagePlugin.JpegImageFile image mode=L size=128x128 at 0x1C83E661C60>, <PIL.JpegImagePlugin.JpegImageFile image mode=L size=128x128 at 0x1C83E661B70>, <PIL.JpegImagePlugin.JpegImageFile image mode=L size=128x128 at 0x1C83E661AE0>], 'label': [2, 0, 3, 3, 2]}


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from datasets import load_dataset
from torchvision import transforms
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import torch.nn.functional as F

In [6]:
torch.manual_seed(42)
np.random.seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device}")

Using cuda


In [11]:


ds = load_dataset("Falah/Alzheimer_MRI")


print(f"Available splits: {list(ds.keys())}")
print(f"Training samples: {len(ds['train'])}")


first_sample = ds['train'][0]

print(f"Keys: {list(first_sample.keys())}")

train_labels = [sample['label'] for sample in ds['train']]
unique_labels, counts = np.unique(train_labels, return_counts=True)

if hasattr(ds['train'].features['label'], 'names'):
    label_names = ds['train'].features['label'].names
    print(f"Label names: {label_names}")
    num_classes = len(label_names)
    for i, (label, count) in enumerate(zip(unique_labels, counts)):
        label_name = label_names[label] if label < len(label_names) else f"Label_{label}"
        print(f"  {label_name}: {count} samples")
else:
    num_classes = len(unique_labels)
    label_names = [f"Class_{i}" for i in range(num_classes)]
    for label, count in zip(unique_labels, counts):
        print(f"  Label {label}: {count} samples")

print(f"Number of classes: {num_classes}")

Available splits: ['train', 'test']
Training samples: 5120
Keys: ['image', 'label']
Label names: ['Mild_Demented', 'Moderate_Demented', 'Non_Demented', 'Very_Mild_Demented']
  Mild_Demented: 724 samples
  Moderate_Demented: 49 samples
  Non_Demented: 2566 samples
  Very_Mild_Demented: 1781 samples
Number of classes: 4


In [24]:
sample_image = first_sample['image']
if hasattr(sample_image, 'size'):
    print(f"Image size: {sample_image.size}")
elif hasattr(sample_image, 'shape'):
    print(f"Image shape: {sample_image.shape}")
else:
    img_array = np.array(sample_image)
    print(f"Image shape: {img_array.shape}")

# Visualize some samples
def visualize_samples(dataset, num_samples=6):
    fig, axes = plt.subplots(2, 3, figsize=(12, 8))
    axes = axes.ravel()
    
    for i in range(num_samples):
        sample = dataset[i]
        image = sample['image']
        label = sample['label']
        
        # Convert to numpy array for visualization
        if hasattr(image, 'numpy'):
            img_array = image.numpy()
        else:
            img_array = np.array(image)

        if len(img_array.shape) == 3 and img_array.shape[0] in [1, 3]:
            img_array = np.transpose(img_array, (1, 2, 0))
        elif len(img_array.shape) == 3 and img_array.shape[2] == 1:
            img_array = img_array.squeeze()
        
        axes[i].imshow(img_array, cmap='gray' if len(img_array.shape) == 2 else None)
        label_name = label_names[label] if label < len(label_names) else f"Label_{label}"
        axes[i].set_title(f'{label_name} (Label: {label})')
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()
    visualize_samples(ds['train'])


Image size: (128, 128)


In [25]:
class AlzheimerMRIDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        sample = self.dataset[idx]
        image = sample['image']
        label = sample['label']
        
        # Convert to RGB if needed
        if hasattr(image, 'convert'):
            if image.mode != 'RGB':
                image = image.convert('RGB')
        
        # Apply transforms
        if self.transform:
            image = self.transform(image)
        else:
            # Default transform
            if not isinstance(image, torch.Tensor):
                image = transforms.ToTensor()(image)
        
        return image, label

In [26]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [27]:
train_dataset = AlzheimerMRIDataset(ds['train'], transform=transform)
from torch.utils.data import random_split
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Apply test transform to validation set
val_dataset.dataset = AlzheimerMRIDataset(ds['train'], transform=test_transform)


In [28]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

In [30]:
print(f"Training batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")

Training batches: 128
Validation batches: 32


In [34]:
class AlzheimerCNN(nn.Module):
    def __init__(self, num_classes):
        super(AlzheimerCNN, self).__init__()
        
        self.conv_layers = nn.Sequential(
            # First conv block
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            # Second conv block
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            # Third conv block
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            # Fourth conv block
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 8 * 8, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )
        
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# Initialize model
model = AlzheimerCNN(num_classes=num_classes).to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\n=== Model Summary ===")
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(loader)
    epoch_acc = 100 * correct / total
    return epoch_loss, epoch_acc

In [ ]:
def validate_epoch(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_predictions = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    epoch_loss = running_loss / len(loader)
    epoch_acc = 100 * correct / total
    return epoch_loss, epoch_acc, all_predictions, all_labels

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
